In [2]:
%matplotlib inline
import sys, os, time, math, csv
import itertools
import collections

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import subprocess

In [3]:
state_ids = []
with open("data/state_fips.csv", "r") as f:
    for line in f:
        line = line.strip()
        if line != "":
            parts = line.split(",")

            fips_code = "%02d" % (int(parts[2]))
            state_ids.append(fips_code)

In [4]:
## Download the block group shapefile for each state from the Census website
for state_id in state_ids:
    subprocess.call([
        "wget",
        "--directory-prefix=data/raw/boundary_shapefiles/block_groups_2012/zips",
        "https://www2.census.gov/geo/tiger/TIGER2012/BG/tl_2012_%s_bg.zip" % (state_id)
    ])

In [5]:
for state_id in state_ids:
    assert os.path.exists("data/raw/boundary_shapefiles/block_groups_2012/zips/tl_2012_%s_bg.zip" % (state_id))

In [6]:
## Unzip all state shapefiles
for state_id in state_ids:
    subprocess.call([
        "unzip",
        os.path.join("data/raw/boundary_shapefiles/block_groups_2012/zips/", "tl_2012_%s_bg.zip" % (state_id)),
        "-d", "data/raw/boundary_shapefiles/block_groups_2012/single_2012/"
    ])

In [7]:
## Merge all shapefiles into single shapefile
for state_id in state_ids:
    subprocess.call([
        "ogr2ogr",
        "-f", "ESRI Shapefile",
        "-update",
        "-append",
        "data/processed/boundary_shapefiles/",
        os.path.join("data/raw/boundary_shapefiles/block_groups_2012/single_2012/", "tl_2012_%s_bg.shp" % (state_id)),
        "-nln", "tl_2012_all_bg"
    ])